# Module Temperature Predictor

Example Use Case for the PV Dataset. Module temperature predictor from environmental measurements and panel characteristics.

## Import libraries

In [63]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor

## Import datasets

In [14]:
pv_dataset = pd.read_csv('../datasets/PV_dataset.csv')
module_metadata = pd.read_csv('../datasets/modules_metadata.csv')

### Get necessary columns for this example:
**Target**: module_temperature_center

**Features**: air_temperature, relative_humidity, wind_speed_ms, G_tilt20_start, G_tilt20_end

In [15]:
pv_dataset['module_name'].value_counts().head()

module_name
CDF1150A1    174105
VBHN330      173922
QPEAK315     173238
LG345N1C     172862
LG370Q1C     172686
Name: count, dtype: int64

For this example, we'll use the CDF1150A1 PV module. 

In [16]:
CDF_metadata = module_metadata.loc[module_metadata['module_name'] == 'CDF1150A1']
NOCT = CDF_metadata['NOCT_c'].values[0]

In [17]:
df_temperature = pv_dataset[['timestamp','module_temperature_center', 'air_temperature', 'relative_humidity', 'wind_speed_ms', 'G_tilt20_start', 'G_tilt20_end']].loc[pv_dataset['module_name'] == 'CDF1150A1']

In [ ]:
# Use average irradiance at Point of Array (POA)
df_temperature['G_POA'] = (df_temperature[['G_tilt20_start', 'G_tilt20_end']].mean(axis=1))
# Get the year
df_temperature['year'] = pd.to_datetime(df_temperature['timestamp']).dt.year

In [19]:
df_temperature.describe()

,module_temperature_center,air_temperature,relative_humidity,wind_speed_ms,G_tilt20_start,G_tilt20_end,G_POA,year
count,173585.000000,174044.000000,173589.000000,173656.000000,174101.000000,174072.000000,174105.000000,174105.000000
mean,33.595143,19.850767,84.110287,1.248791,402.003415,392.926126,397.438484,2023.529353
std,10.842636,3.073996,6.631206,0.888443,303.240084,302.486700,302.613256,1.114974
min,1.960000,12.950000,64.730000,0.000000,25.010000,13.210000,19.345000,2022.000000
25%,24.880000,17.300000,79.320000,0.620000,143.210000,134.570000,138.870000,2023.000000
50%,31.420000,19.790000,83.950000,1.030000,312.680000,303.355000,308.065000,2024.000000
75%,42.120000,22.050000,89.010000,1.720000,630.670000,621.440000,626.585000,2025.000000
max,64.540000,29.240000,100.000000,4.970000,1982.440000,1380.950000,1488.540000,2025.000000


## Define features and target and split

In [28]:
df_temperature.drop(['G_tilt20_start', 'G_tilt20_end'], axis=1).isna().sum()

timestamp                      0
module_temperature_center    520
air_temperature              963
relative_humidity            963
wind_speed_ms                963
G_POA                        963
year                           0
dtype: int64

We'll train on data from 2022-2024 and test on 2025

In [29]:
df_temperature['year'].value_counts()

year
2025    44696
2024    44607
2023    42966
2022    41836
Name: count, dtype: int64

In [30]:
features = ['air_temperature', 'relative_humidity', 'wind_speed_ms', 'G_POA']

NaN values represent a small portion of the dataset. In this case, we'll discard them.

In [31]:
df_temperature.dropna(inplace=True)

In [32]:
X_train = df_temperature[df_temperature['year'] < 2025][features]
y_train = df_temperature[df_temperature['year'] < 2025]['module_temperature_center']
X_test = df_temperature[df_temperature['year'] == 2025][features]
y_test = df_temperature[df_temperature['year'] == 2025]['module_temperature_center']

# AI Models

# Baseline - NOCT model

In [33]:
T_module_baseline = X_test['air_temperature'] + (NOCT - 20) * X_test['G_POA'] / 800

In [60]:
root_mse = mean_squared_error(y_test, T_module_baseline) ** 0.5
print(f"Baseline RMSE using NOCT formula: {root_mse:.2f} °C" )

Baseline RMSE using NOCT formula: 4.90 °C


## Linear Regression

In [57]:
reg = LinearRegression().fit(X_train, y_train)
reg.score(X_test, y_test)
pred_reg = reg.predict(X_test)

In [59]:
rmse_reg = mean_squared_error(y_test, pred_reg) ** 0.5
print(f"RMSE using Linear Regression: {rmse_reg:.2f} °C" )

RMSE using Linear Regression: 4.75 °C


## Random Forest Regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=100, random_state=18)

rf.fit(X_train, y_train)
pred_rf = rf.predict(X_test)

In [ ]:
rmse_rf = mean_squared_error(y_test, pred_rf) ** 0.5
print(f"RMSE using Random Forest: {rmse_rf:.2f} °C" )

RMSE using Random Forest: 4.70 °C


## Gradient Boost

In [64]:
gbr = GradientBoostingRegressor(n_estimators=500, learning_rate=0.05, max_depth=3, random_state=18)
gbr.fit(X_train, y_train)
pred_gbr = gbr.predict(X_test)

In [65]:
rmse_gbr = mean_squared_error(y_test, pred_gbr) ** 0.5
print(f"RMSE using Gradient Boosting: {rmse_gbr:.2f} °C" )

RMSE using Gradient Boosting: 4.62 °C


# Summary

In [89]:
results_dict = {'Linear Regression': rmse_reg, 'Random_Forest': rmse_rf, 'Gradient Boosting': rmse_gbr}
results_df = pd.DataFrame.from_dict(results_dict, orient='index', columns=['RMSE (°C)'])
results_df

,RMSE (°C)
Linear Regression,4.746429
Random_Forest,4.698478
Gradient Boosting,4.615819
